In [45]:
from tensorflow import keras
import os
import h5py
import numpy as np
#print(os.getcwd())

In [46]:
def load_OASIS():
    with h5py.File('..\..\Datasets\OASIS_balanced.h5', 'r') as hdf:

        G1 = hdf.get('Train Data')
        trainX = np.array(G1.get('trainX'))
        trainY = np.array(G1.get('trainY'))
        G2 = hdf.get('Test Data')
        testX = np.array(G2.get('testX'))
        testY = np.array(G2.get('testY'))

        return trainX, trainY, testX, testY

In [47]:
def load_ADNI():
    with h5py.File('..\..\Datasets\ADNI_enhanced.h5', 'r') as hdf:

        G1 = hdf.get('Train Data')
        trainX = np.array(G1.get('x_train'))
        trainY = np.array(G1.get('y_train'))
        G2 = hdf.get('Test Data')
        testX = np.array(G2.get('x_test'))
        testY = np.array(G2.get('y_test'))

        return trainX, trainY, testX, testY

In [57]:
dataset = 'ADNdI'
#dataset = 'ADsNI'
# read the data which is also normalized.
if dataset == 'ADNI':
    x_train, y_train, x_test, y_test = load_ADNI()
else:
    x_train, y_train, x_test, y_test = load_OASIS()

print(x_train.shape, y_train.shape, x_test.shape,y_test.shape)

(8192, 176, 176) (8192,) (2560, 176, 176) (2560,)


In [58]:
from tensorflow.keras.utils import to_categorical
x_train = np.expand_dims(x_train, axis=-1)
x_train = np.repeat(x_train, 3, axis=3)

x_test = np.expand_dims(x_test, axis=-1)
x_test = np.repeat(x_test, 3, axis=3)

y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat = to_categorical(y_test, num_classes=4)
print('X train shape', x_train.shape)
print('Y train shape', y_train_cat.shape)
row = x_train.shape[1]
col = x_train.shape[2]
print(row, col)

X train shape (8192, 176, 176, 3)
Y train shape (8192, 4)
176 176


In [53]:
#load model-A (CNN from scratch) architecture and weights 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

modelA = Sequential()

modelA.add(Conv2D(16, (3, 3), input_shape=(176,176,1), activation='relu'))
modelA.add(Conv2D(16, (3, 3), activation='relu'))
modelA.add(MaxPooling2D(pool_size=(2, 2)))

modelA.add(Conv2D(32, (3, 3), activation='relu'))
modelA.add(Conv2D(32, (3, 3), activation='relu'))
modelA.add(Conv2D(32, (3, 3), activation='relu'))
modelA.add(MaxPooling2D(pool_size = (2, 2)))

modelA.add(Conv2D(64, (3, 3), activation='relu'))
modelA.add(Conv2D(16, (1,1)))

modelA.add(Flatten())

modelA.add(Dense(4, activation = 'softmax'))

modelA.compile(loss='categorical_crossentropy',optimizer='adam',metrics =['acc'])
modelA.summary()

modelA.load_weights('A.hdf5')

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1136 (Conv2D)         (None, 174, 174, 16)      160       
_________________________________________________________________
conv2d_1137 (Conv2D)         (None, 172, 172, 16)      2320      
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 86, 86, 16)        0         
_________________________________________________________________
conv2d_1138 (Conv2D)         (None, 84, 84, 32)        4640      
_________________________________________________________________
conv2d_1139 (Conv2D)         (None, 82, 82, 32)        9248      
_________________________________________________________________
conv2d_1140 (Conv2D)         (None, 80, 80, 32)        9248      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 40, 40, 32)      

In [41]:
#load model-B (InceptionV3) architecture and weights 
import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization
modelB = Sequential()

pretrained_model= tf.keras.applications.InceptionV3(include_top=False,
                   input_shape=(176,176,3),
                   #input_shape=(218,182,3),
                   pooling='avg',classes=4,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

modelB.add(pretrained_model)
modelB.add(Dropout(0.5))
modelB.add(Flatten())
modelB.add(BatchNormalization())
modelB.add(Dense(2048,kernel_initializer='he_uniform'))
modelB.add(BatchNormalization())
modelB.add(Activation('relu'))
modelB.add(Dropout(0.5))
modelB.add(Dense(1024,kernel_initializer='he_uniform'))
modelB.add(BatchNormalization())
modelB.add(Activation('relu'))
modelB.add(Dropout(0.5))
modelB.add(Dense(4,activation='softmax'))
modelB.summary()

modelB.load_weights('B.hdf5')

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dropout_16 (Dropout)         (None, 2048)              0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 2048)              0         
_________________________________________________________________
batch_normalization_906 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_21 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_907 (Bat (None, 2048)              8192      
_________________________________________________________________
activation_901 (Activation)  (None, 2048)            

In [42]:
#load model-C (InceptionResNetV2) architecture and weights 
import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization
modelC = Sequential()

pretrained_model= tf.keras.applications.InceptionResNetV2(include_top=False,
                   input_shape=(176,176,3),
                   #input_shape=(218,182,3),
                   pooling='avg',classes=4,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

modelC.add(pretrained_model)
modelC.add(Dropout(0.5))
modelC.add(Flatten())
modelC.add(BatchNormalization())
modelC.add(Dense(2048,kernel_initializer='he_uniform'))
modelC.add(BatchNormalization())
modelC.add(Activation('relu'))
modelC.add(Dropout(0.5))
modelC.add(Dense(1024,kernel_initializer='he_uniform'))
modelC.add(BatchNormalization())
modelC.add(Activation('relu'))
modelC.add(Dropout(0.5))
modelC.add(Dense(4,activation='softmax'))
modelC.summary()

modelC.load_weights('C.hdf5')

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dropout_19 (Dropout)         (None, 1536)              0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 1536)              0         
_________________________________________________________________
batch_normalization_1112 (Ba (None, 1536)              6144      
_________________________________________________________________
dense_24 (Dense)             (None, 2048)              3147776   
_________________________________________________________________
batch_normalization_1113 (Ba (None, 2048)              8192      
_________________________________________________________________
activation_1106 (Activation) (None, 2048)            

In [43]:
#load model-D (ResNet101) architecture and weights 

import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization
modelD = Sequential()

pretrained_model= tf.keras.applications.ResNet101(include_top=False,
                   input_shape=(176,176,3),
                   #input_shape=(218,182,3),
                   pooling='avg',classes=4,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

modelD.add(pretrained_model)
modelD.add(Dropout(0.5))
modelD.add(Flatten())
modelD.add(BatchNormalization())
modelD.add(Dense(2048,kernel_initializer='he_uniform'))
modelD.add(BatchNormalization())
modelD.add(Activation('relu'))
modelD.add(Dropout(0.5))
modelD.add(Dense(1024,kernel_initializer='he_uniform'))
modelD.add(BatchNormalization())
modelD.add(Activation('relu'))
modelD.add(Dropout(0.5))
modelD.add(Dense(4,activation='softmax'))
modelD.summary()

modelD.load_weights('D.hdf5')

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 2048)              42658176  
_________________________________________________________________
dropout_22 (Dropout)         (None, 2048)              0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 2048)              0         
_________________________________________________________________
batch_normalization_1115 (Ba (None, 2048)              8192      
_________________________________________________________________
dense_27 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_1116 (Ba (None, 2048)              8192      
_________________________________________________________________
activation_1108 (Activation) (None, 2048)            

In [62]:
x_traina, y_traina, x_testa, y_testa = load_OASIS()
x_traina = np.expand_dims(x_traina,axis = -1)
x_testa = np.expand_dims(x_testa,axis = -1)

modelAprediction = modelA.predict(x_testa)

In [63]:
print(modelAprediction.shape)

(2560, 4)


In [65]:
modelBprediction = modelB.predict(x_test)


In [66]:
modelCprediction = modelC.predict(x_test)

In [67]:
modelDprediction = modelD.predict(x_test)

In [81]:
#ensemble prediction 
preds = np.stack([modelAprediction,modelBprediction,modelCprediction, modelDprediction], axis=0)

summed = np.sum(preds, axis=0) #computes the sum of probability by all 2 models for each class
ensemble_prediction = np.argmax(summed, axis=1) 
ensemble_accuracy = accuracy_score(y_test, ensemble_prediction)
print('ensemble accuracy =', ensemble_accuracy)

ensemble accuracy = 0.965625


In [55]:
#Ensemble of 2 models
model1, model2 = modelB, modelC 
models = [modelB, modelC]
print ("prediction by all models")
preds = [model.predict(x_test) for model in models]
preds=np.array(preds)
print(preds.shape)
summed = np.sum(preds, axis=0) #computes the sum of probability by all 2 models for each class

ensemble_prediction = np.argmax(summed, axis=1) #predicts the class labels based on maximum probability.

print("predicting classes by individual models")
prediction1 = model1.predict_classes(x_test)
prediction2 = model2.predict_classes(x_test)

print("computing accuracy by all models")
from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(y_test, prediction1)
accuracy2 = accuracy_score(y_test, prediction2)
ensemble_accuracy = accuracy_score(y_test, ensemble_prediction)


print('Accuracy Score for model1 = ', accuracy1)
print('Accuracy Score for model2 = ', accuracy2)
print('ensemble accuracy =', ensemble_accuracy)

prediction by all models
(2, 2560, 4)
predicting classes by individual models


c:\Users\noush\anaconda3\envs\tf2.5\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


computing accuracy by all models
Accuracy Score for model1 =  0.912890625
Accuracy Score for model2 =  0.90078125
ensemble accuracy = 0.936328125
